In [ ]:
%load_ext autoreload
%autoreload 2

In [80]:
import SimulateData

In [4]:
config = {
    "signatures_file_path":     'cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt',
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'],
    "n_samples":                10,
    "save_dir":                 'simulated_data',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':     True,
        'sign_active_prob':         0.4,    # Only used if use_sign_active_prob is True
        'n_sign_active':            2       # Only used if use_sign_active_prob is False
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg_perc':                 0.05,
    },
    'counts_distribution': {
        'cancer_type':              'random', # Specify the cancer_type, or let each sample be a random one from the file 'mutation_counts/TCGA/WES_TCGA.96_min_max.csv'
        # 'cancer_type':              'NA',   # If want to not use the cancertype to get the min and max, but fill them in by hand
        'distribution':             'logscale',
        # 'min':                      1000,   # Only used if cancer_type = NA
        # 'max':                      50000,  # Only used if cancer_type = NA
    }
}

KeyError: 'identifier'

In [ ]:
!ls simulated_data

In [83]:
import pandas as pd
import random
import re
signatures = pd.read_csv('cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt', sep='\t')
signatures.set_index('Type', inplace=True)
signatures = signatures[[x for x in signatures.columns if not re.search(r'SBS5$|SBS40.?$',x)]]

import numpy as np


orthogonal = dict()
for col_name, col_data in signatures.items():
    other = signatures.drop(columns=col_name)
    other_products = other.apply(lambda x: np.dot(col_data, x))
    orthogonal[col_name] = other_products.mean()

orthogonal = pd.Series(orthogonal).sort_values(ascending=False)

In [84]:
# for signature_count in [8, 15, 25]:
#     signature_list = " ".join(orthogonal[:signature_count].index)
#     for noise_level in [0.02, 0.04, 0.06]:
#         for mutation_count in [(500, 2000), (2000, 5000), (5000, 8000)]:
#             print(signature_list)
#             identifier = f"s_{signature_count}_n_{noise_level}"
#             !python SimulateData.py --identifier {identifier} --config "simulated_data/config_v1.json" --signatures_to_extract $signature_list --noise_distribution_avg_perc $noise_level --counts_distribution_min $mutation_count[0] --counts_distribution_max $mutation_count[1]
#             break

In [ ]:
for signature_count in [8, 15, 25]:
    signature_list = " ".join(orthogonal[:signature_count].index)
    for noise_level in [0.02, 0.04, 0.06]:
        print(signature_list)
        identifier = f"s_{signature_count}_n_{noise_level}"
        !python SimulateData.py --identifier {identifier} --config "simulated_data/config_v2.json" --signatures_to_extract $signature_list --noise_distribution_avg_perc $noise_level --counts_distribution_min
        break

In [ ]:
# Quick test
# !python SimulateData.py --identifier "352" --config "simulated_data/config_v2.json" --signatures_to_extract $signature_list --noise_distribution_avg_perc $noise_level